In [9]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


import numpy as np
#from matting import load_path,load_data,load_alphamatting_data,load_validation_data
import os
from scipy import misc
#os.environ['CUDA_VISIBLE_DEVICES']='0'

In [10]:
image_size = 320
train_batch_size = 1
max_epochs = 1000000
hard_mode = False

#checkpoint file path
pretrained_model = False
#pretrained_model = False
test_dir = './alhpamatting'
test_outdir = './test_predict'
#validation_dir = '/data/gezheng/data-matting/new2/validation'

#pretrained_vgg_model_path
# model_path = './vgg16_weights.npz'
log_dir = 'matting_log'

dataset_alpha = 'train_data/alpha'
dataset_eps = 'train_data/eps'
dataset_BG = 'train_data/bg'

paths_alpha,paths_eps,paths_BG = load_path(dataset_alpha,dataset_eps,dataset_BG,hard_mode = hard_mode)

range_size = len(paths_alpha)
print('range_size is %d' % range_size)
#range_size/batch_size has to be int
batchs_per_epoch = int(range_size/train_batch_size) 

NameError: name 'load_path' is not defined

<h2>!!!!!!!!!</h2>

In [11]:
"""
index_queue = tf.train.range_input_producer(range_size, num_epochs=None,shuffle=True, seed=None, capacity=32)
index_dequeue_op = index_queue.dequeue_many(train_batch_size, 'index_dequeue')
"""


"""
image_batch = tf.placeholder(tf.float32, shape=(train_batch_size,image_size,image_size,3))
raw_RGBs = tf.placeholder(tf.float32, shape=(train_batch_size,image_size,image_size,3))
GT_matte_batch = tf.placeholder(tf.float32, shape = (train_batch_size,image_size,image_size,1))
GT_trimap = tf.placeholder(tf.float32, shape = (train_batch_size,image_size,image_size,1))
GTBG_batch = tf.placeholder(tf.float32, shape = (train_batch_size,image_size,image_size,3))
GTFG_batch = tf.placeholder(tf.float32, shape = (train_batch_size,image_size,image_size,3))
training = tf.placeholder(tf.bool)
"""



"""
tf.add_to_collection('image_batch',image_batch)
tf.add_to_collection('GT_trimap',GT_trimap)
tf.add_to_collection('training',training)
"""

"""
en_parameters = []
pool_parameters = []
"""


b_RGB = tf.identity(image_batch,name = 'b_RGB')
b_trimap = tf.identity(GT_trimap,name = 'b_trimap')
b_GTmatte = tf.identity(GT_matte_batch,name = 'b_GTmatte')
b_GTBG = tf.identity(GTBG_batch,name = 'b_GTBG')
b_GTFG = tf.identity(GTFG_batch,name = 'b_GTFG')

tf.summary.image('GT_matte_batch',b_GTmatte,max_outputs = 5)
tf.summary.image('trimap',b_trimap,max_outputs = 5)
tf.summary.image('raw_RGBs',raw_RGBs,max_outputs = 5)

b_input = tf.concat([b_RGB,b_trimap],3)

NameError: name 'tf' is not defined

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        activation = torch.nn.ReLU()
        # conv1_1  
        conv1_1 = torch.nn.Conv2d(in_channels = 4, 
                       out_channels = 64,
                       kernel_size = [3, 3])
        # conv1_2
        conv1_2 = torch.nn.Conv2d(in_channels = 64, 
                       out_channels = 64,
                       kernel_size = [3, 3])
        # pool1
        pool1 = torch.nn.MaxPool2d(kernel_size = [2, 2], 
                       stride = [2, 2])
        # conv2_1
        conv2_1 = torch.nn.Conv2d(in_channels = 64, 
                       out_channels = 128,
                       kernel_size = [3, 3])
        # conv2_2
        conv2_2 = torch.nn.Conv2d(in_channels = 128, 
                       out_channels = 128,
                       kernel_size = [3, 3])
        # pool2
        pool2 = torch.nn.MaxPool2d(kernel_size = [2, 2], 
                       stride = [2, 2])
        # conv3_1
        conv3_1 = torch.nn.Conv2d(in_channels = 128, 
                       out_channels = 256,
                       kernel_size = [3, 3])
        # conv3_2
        conv3_2 = torch.nn.Conv2d(in_channels = 256, 
                       out_channels = 256,
                       kernel_size = [3, 3])
        # conv3_3
        conv3_3 = torch.nn.Conv2d(in_channels = 256, 
                       out_channels = 256,
                       kernel_size = [3, 3])
        # pool3
        pool3 = torch.nn.MaxPool2d(kernel_size = [2, 2], 
                       stride = [2, 2])
        # conv4_1
        conv4_1 = torch.nn.Conv2d(in_channels = 256, 
                       out_channels = 512,
                       kernel_size = [3, 3])
        # conv4_2
        conv4_2 = torch.nn.Conv2d(in_channels = 512, 
                       out_channels = 512,
                       kernel_size = [3, 3])
        # conv4_3
        conv4_3 = torch.nn.Conv2d(in_channels = 512, 
                       out_channels = 512,
                       kernel_size = [3, 3])
        # pool4
        pool4 = torch.nn.MaxPool2d(kernel_size = [2, 2], 
                       stride = [2, 2])
        # conv5_1
        conv5_1 = torch.nn.Conv2d(in_channels = 512, 
                       out_channels = 512,
                       kernel_size = [3, 3])
        # conv5_2
        conv5_2 = torch.nn.Conv2d(in_channels = 512, 
                       out_channels = 512,
                       kernel_size = [3, 3])
        # conv5_3
        conv5_3 = torch.nn.Conv2d(in_channels = 512, 
                       out_channels = 512,
                       kernel_size = [3, 3])
        # pool5
        pool5 = torch.nn.MaxPool2d(kernel_size = [2, 2], 
                       stride = [2, 2])
        # conv6_1 
        conv6_1 = torch.nn.Conv2d(in_channels = 512, 
                       out_channels = 4046,
                       kernel_size = [7, 7])
        
        
        # deconv6
        deconv6 = torch.nn.ConvTranspose2d(in_channels = 4096, 
                             out_channels = 512, 
                             kernel_size = [1, 1])
        batch_norm_6 = torch.nn.BatchNorm2d(num_features = 512)
        # deconv5_1/unpooling
        unpool_5 = torch.nn.MaxUnpool2d(kernel_size = [2, 2], 
                              stride = [2, 2])
        # deconv5_2
        deconv5_2 = torch.nn.ConvTranspose2d(in_channels = 512, 
                             out_channels = 512, 
                             kernel_size = [5, 5])
        batch_norm_5 = torch.nn.BatchNorm2d(num_features = 512)
        # deconv4_1/unpooling
        unpool_4 = torch.nn.MaxUnpool2d(kernel_size = [2, 2], 
                              stride = [2, 2])
        # deconv4_2
        deconv4_2 = torch.nn.ConvTranspose2d(in_channels = 512, 
                             out_channels = 256, 
                             kernel_size = [5, 5])
        batch_norm_4 = torch.nn.BatchNorm2d(num_features = 256)
        # deconv3_1/unpooling
        unpool_3 = torch.nn.MaxUnpool2d(kernel_size = [2, 2], 
                       stride = [2, 2])
        # deconv3_2
        deconv3_2 = torch.nn.ConvTranspose2d(in_channels = 256, 
                             out_channels = 128, 
                             kernel_size = [5, 5])
        batch_norm_3 = torch.nn.BatchNorm2d(num_features = 128)
        # deconv2_1/unpooling
        unpool_2 = torch.nn.MaxUnpool2d(kernel_size = [2, 2], 
                       stride = [2, 2])
        # deconv2_2
        deconv2_2 = torch.nn.ConvTranspose2d(in_channels = 128, 
                             out_channels = 64, 
                             kernel_size = [5, 5])
        batch_norm_2 = torch.nn.BatchNorm2d(num_features = 64)
        # deconv1_1/unpooling
        unpool_1 = torch.nn.MaxUnpool2d(kernel_size = [2, 2], 
                       stride = [2, 2])
        # deconv1_2    
        deconv1_2 = torch.nn.ConvTranspose2d(in_channels = 64, 
                             out_channels = 64, 
                             kernel_size = [5, 5])
        batch_norm_1 = torch.nn.BatchNorm2d(num_features = 64)
        # pred_alpha_matte
        pred_alpha_matte = torch.nn.Conv2d(in_channels = 64, 
                        out_channels = 1, 
                        kernel_size = [5, 5])
        act_sigmoid = torch.nn.Sigmoid()

                                 
                                 
        
    def forward(self, b_input):
        # conv1_1  
        x = conv1_1(b_input)
        x = actiovation(x)
            
        # conv1_2
        x = conv1_2(x)
        x = actiovation(x)
            
        # pool1
        x = pool1(x)
            
        # conv2_1
        x = conv2_1(x)
        x = actiovation(x)
            
        # conv2_2
        x = conv2_2(x)
        x = actiovation(x)
            
        # pool2
        x = pool2(x)
            
        # conv3_1
        x = conv3_1(x)
        x = actiovation(x)
            
        # conv3_2
        x = conv3_2(x)
        x = actiovation(x)
            
        # conv3_3
        x = conv3_3(x)
        x = actiovation(x)
            
        # pool3
        x = pool3(x)

        # conv4_1
        x = conv4_1(x)
        x = actiovation(x)
        
        # conv4_2
        x = conv4_2(x)
        x = actiovation(x)
        
        # conv4_3
        x = conv4_3(x)
        x = actiovation(x)
        
        # pool4
        x = pool4(x)
        
        # conv5_1
        x = conv5_1(x)
        x = actiovation(x)
        
        # conv5_2
        x = conv5_2(x)
        x = actiovation(x)
        
        # conv5_3
        x = conv5_3(x)
        x = actiovation(x)
        
        # pool5
        x = pool5(x)
        
        # conv6_1 
        x = conv6_1(x)
        x = actiovation(x)
        
        
        # deconv6
        x = deconv6(x)
        x = batch_norm_6(x)
        x = activation(x)
        
        #deconv5_1/unpooling
        x = unpool_5(x)
                                            
        # deconv5_2
        x = deconv5_2(x)
        x = batch_norm_5(x)
        x = activation(x)
        
        # deconv4_1/unpooling)
        x = unpool_4(x)
        
        # deconv4_2
        x = deconv4_2(x)
        x = batch_norm_4(x)
        x = activation(x)
        
        # deconv3_1/unpooling
        x = unpool_3(x)
        
        # deconv3_2
        x = deconv3_2(x)
        x = batch_norm_3(x)
        x = activation(x)
        
        # deconv2_1/unpooling
        x = unpool_2(x)
        
        # deconv2_2
        x = deconv2_2(x)
        x = batch_norm_2(x)
        x = activation(x)
        
        # deconv1_1/unpooling
        x = unpool_1(x)
        
        # deconv1_2  
        x = deconv1_2(x)
        x = batch_norm_1(x)
        x = activation(x)
        
        # pred_alpha_matte
        x = pred_alpha_matte(x)
        pred_mattes = act_sigmoid(x)
        
        return pred_mattes
        
        

<h3> create net</h3>

In [13]:
net = Net()

<h3>define loss and optimizer</h3>

In [ ]:
def loss():
    wl = torch.where(torch.equal(b_trimap, 128), 
                 torch.Tensor([train_batch_size,image_size,image_size,1]).fill(1.),
                 torch.Tensor([train_batch_size,image_size,image_size,1]).fill(0.))

    unknown_region_size = torch.sum(wl)

    alpha_diff = torch.sqrt((pred_mattes - GT_matte_batch)**2 + 1e-12)

    p_RGB = []

    b_GTBG = b_GTBG.view([train_batch_size,image_size,image_size,3])
    b_GTFG = b_GTFG.view([train_batch_size,image_size,image_size,3])

    raw_RGBs = raw_RGBs.view([train_batch_size,image_size,image_size,3])
    b_GTmatte = b_GTmatte.view([train_batch_size,image_size,image_size,1])
    
    pred_final = torch.where(torch.equal(b_trimap, 128), , pred_mattes,b_trimap/255.0)

    l_matte = torch.unbind(pred_final)
    BG = torch.unbind(b_GTBG)
    FG = torch.unbind(b_GTFG)

    for i in range(train_batch_size):
        p_RGB.append(l_matte[i] * FG[i] + (1. - l_matte[i]) * BG[i])
    pred_RGB = torch.stack(p_RGB)

    c_diff = torch.sqrt((pred_RGB - raw_RGBs)**2 + 1e-12)/255.0

    alpha_loss = torch.sum(alpha_diff * wl)/(unknown_region_size)
    comp_loss = torch.sum(c_diff * wl)/(unknown_region_size)

    total_loss = (alpha_loss + comp_loss) * 0.5
    
    return total_loss


optimizer = torch.optim.Adam(net.parameters(), lr=1e-5)
criterion = loss()

loss.backward()

<h3>train</h3>

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    
    while epoch_num < max_epochs:  
        while batch_num < batchs_per_epoch:
            batch_alpha_paths = paths_alpha[batch_index]
            batch_eps_paths = paths_eps[batch_index]
            batch_BG_paths = paths_BG[batch_index]
            batch_RGBs,batch_trimaps,batch_alphas,batch_BGs,batch_FGs,RGBs_with_mean = load_data(batch_alpha_paths,batch_eps_paths,batch_BG_paths)

            
            loss.backward()
            optimizer.step()

In [ ]:
with tf.Session(config=tf.ConfigProto(gpu_options = gpu_options)) as sess:

    while epoch_num < max_epochs:  
        while batch_num < batchs_per_epoch:

            batch_alpha_paths = paths_alpha[batch_index]
            batch_eps_paths = paths_eps[batch_index]
            batch_BG_paths = paths_BG[batch_index]
            batch_RGBs,batch_trimaps,batch_alphas,batch_BGs,batch_FGs,RGBs_with_mean = load_data(batch_alpha_paths,batch_eps_paths,batch_BG_paths)

            feed = {image_batch:batch_RGBs, GT_matte_batch:batch_alphas,GT_trimap:batch_trimaps, GTBG_batch:batch_BGs, GTFG_batch:batch_FGs,raw_RGBs:RGBs_with_mean,training:True}

            _,loss,summary_str,step= sess.run([train_op,total_loss,summary_op,global_step],feed_dict = feed)
            print('epoch %d   batch %d   loss is %f' %(epoch_num,batch_num,loss))

            if step%200 == 0:
                print('saving model......')
                saver.save(sess,'./model/model.ckpt',global_step = step, write_meta_graph = False)

                print('test on validation data...')
                test_RGBs,test_trimaps,test_alphas,all_shape,image_paths,trimap_size = load_alphamatting_data(test_dir)
                vali_diff = []
                
                for i in range(len(test_RGBs)):
                    test_RGB = np.expand_dims(test_RGBs[i],0)
                    test_trimap = np.expand_dims(test_trimaps[i],0)
                    test_alpha = test_alphas[i]
                    shape_i = all_shape[i]
                    image_path = image_paths[i]
                    
                    feed = {image_batch:test_RGB,GT_trimap:test_trimap,training:False}
                    test_out = sess.run(pred_final,feed_dict = feed)
                    
                    i_out = misc.imresize(test_out[0,:,:,0],shape_i)
                    vali_diff.append(np.sum(np.abs(i_out/255.0-test_alpha))/trimap_size[i])
                    misc.imsave(os.path.join(test_outdir,image_path),i_out)
                
                vali_loss = np.mean(vali_diff)
                print('validation loss is '+ str(vali_loss))
                validation_summary = tf.Summary()
                validation_summary.value.add(tag='validation_loss',simple_value = vali_loss)
                summary_writer.add_summary(validation_summary,step)

            summary_writer.add_summary(summary_str,global_step = step)
            batch_num += 1
        batch_num = 0
        epoch_num += 1

In [ ]:
tf.add_to_collection("pred_mattes", pred_mattes)

wl = tf.where(tf.equal(b_trimap,128),tf.fill([train_batch_size,image_size,image_size,1],1.),tf.fill([train_batch_size,image_size,image_size,1],0.))
unknown_region_size = tf.reduce_sum(wl)

tf.summary.image('pred_mattes',pred_mattes,max_outputs = 5)
alpha_diff = tf.sqrt(tf.square(pred_mattes - GT_matte_batch)+ 1e-12)

p_RGB = []
pred_mattes.set_shape([train_batch_size,image_size,image_size,1])
b_GTBG.set_shape([train_batch_size,image_size,image_size,3])
b_GTFG.set_shape([train_batch_size,image_size,image_size,3])
raw_RGBs.set_shape([train_batch_size,image_size,image_size,3])
b_GTmatte.set_shape([train_batch_size,image_size,image_size,1])

pred_final =  tf.where(tf.equal(b_trimap,128), pred_mattes,b_trimap/255.0)
tf.summary.image('pred_final',pred_final,max_outputs = 5)

l_matte = tf.unstack(pred_final)
BG = tf.unstack(b_GTBG)
FG = tf.unstack(b_GTFG)

for i in range(train_batch_size):
    p_RGB.append(l_matte[i] * FG[i] + (tf.constant(1.) - l_matte[i]) * BG[i])
pred_RGB = tf.stack(p_RGB)

tf.summary.image('pred_RGB',pred_RGB,max_outputs = 5)
c_diff = tf.sqrt(tf.square(pred_RGB - raw_RGBs) + 1e-12)/255.0

alpha_loss = tf.reduce_sum(alpha_diff * wl)/(unknown_region_size)
comp_loss = tf.reduce_sum(c_diff * wl)/(unknown_region_size)

# tf.summary.image('alpha_diff',alpha_diff * wl_alpha,max_outputs = 5)
# tf.summary.image('c_diff',c_diff * wl_RGB,max_outputs = 5)

tf.summary.scalar('alpha_loss',alpha_loss)
tf.summary.scalar('comp_loss',comp_loss)

total_loss = (alpha_loss + comp_loss) * 0.5
tf.summary.scalar('total_loss',total_loss)
global_step = tf.Variable(0,trainable=False)


train_op = tf.train.AdamOptimizer(learning_rate = 1e-5).minimize(total_loss,global_step = global_step)

saver = tf.train.Saver(tf.trainable_variables() , max_to_keep = 1)

coord = tf.train.Coordinator()
summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.6)
with tf.Session(config=tf.ConfigProto(gpu_options = gpu_options)) as sess:
    sess.run(tf.global_variables_initializer())
    tf.train.start_queue_runners(coord=coord,sess=sess)
    batch_num = 0
    epoch_num = 0
    #initialize all parameters in vgg16
    if not pretrained_model:
        weights = np.load(model_path)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            if i == 28:
                break
            if k == 'conv1_1_W':  
                sess.run(en_parameters[i].assign(np.concatenate([weights[k],np.zeros([3,3,1,64])],axis = 2)))
            else:
                if k=='fc6_W':
                    tmp = np.reshape(weights[k],(7,7,512,4096))
                    sess.run(en_parameters[i].assign(tmp))
                else:
                    sess.run(en_parameters[i].assign(weights[k]))
        print('finish loading vgg16 model')
    else:
        print('Restoring pretrained model...')
        saver.restore(sess,tf.train.latest_checkpoint('./model'))
    sess.graph.finalize()

    while epoch_num < max_epochs:  
        while batch_num < batchs_per_epoch:
            batch_index = sess.run(index_dequeue_op)

            batch_alpha_paths = paths_alpha[batch_index]
            batch_eps_paths = paths_eps[batch_index]
            batch_BG_paths = paths_BG[batch_index]
            batch_RGBs,batch_trimaps,batch_alphas,batch_BGs,batch_FGs,RGBs_with_mean = load_data(batch_alpha_paths,batch_eps_paths,batch_BG_paths)

            feed = {image_batch:batch_RGBs, GT_matte_batch:batch_alphas,GT_trimap:batch_trimaps, GTBG_batch:batch_BGs, GTFG_batch:batch_FGs,raw_RGBs:RGBs_with_mean,training:True}

            _,loss,summary_str,step= sess.run([train_op,total_loss,summary_op,global_step],feed_dict = feed)
            print('epoch %d   batch %d   loss is %f' %(epoch_num,batch_num,loss))

            if step%200 == 0:
                print('saving model......')
                saver.save(sess,'./model/model.ckpt',global_step = step, write_meta_graph = False)

                print('test on validation data...')
                test_RGBs,test_trimaps,test_alphas,all_shape,image_paths,trimap_size = load_alphamatting_data(test_dir)
                vali_diff = []
                
                for i in range(len(test_RGBs)):
                    test_RGB = np.expand_dims(test_RGBs[i],0)
                    test_trimap = np.expand_dims(test_trimaps[i],0)
                    test_alpha = test_alphas[i]
                    shape_i = all_shape[i]
                    image_path = image_paths[i]
                    
                    feed = {image_batch:test_RGB,GT_trimap:test_trimap,training:False}
                    test_out = sess.run(pred_final,feed_dict = feed)
                    
                    i_out = misc.imresize(test_out[0,:,:,0],shape_i)
                    vali_diff.append(np.sum(np.abs(i_out/255.0-test_alpha))/trimap_size[i])
                    misc.imsave(os.path.join(test_outdir,image_path),i_out)
                
                vali_loss = np.mean(vali_diff)
                print('validation loss is '+ str(vali_loss))
                validation_summary = tf.Summary()
                validation_summary.value.add(tag='validation_loss',simple_value = vali_loss)
                summary_writer.add_summary(validation_summary,step)

            summary_writer.add_summary(summary_str,global_step = step)
            batch_num += 1
        batch_num = 0
        epoch_num += 1